In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard, LambdaCallback
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict
import jpeg4py as jpeg
from io import BytesIO

Using TensorFlow backend.


In [2]:
#parameters
classes = 10
batch_size=50
image_size = 224
train_total = 3500
validation_total = 4370

In [3]:
model_densenet = DenseNet201(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3))

In [5]:
x = model_densenet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
for layer in model_densenet.layers:
    layer.trainable = True

In [8]:
model = Model(inputs=model_densenet.input, outputs=x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [9]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [10]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )

Found 790600 images belonging to 10 classes.


In [10]:
validation_generator = validation_datagen.flow_from_directory(
                        '../validation_224/',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 4370 images belonging to 10 classes.


In [11]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    del image
    return image_crop

In [12]:
def random_crop_valid(image, crop_size):
    h,w,d = image.shape
    image_crop = image[h//2-crop_size//2:h//2+crop_size//2,w//2-crop_size//2:w//2+crop_size//2,:]
    
    return image_crop

In [13]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
    
                for i in range(start,end):
                    image_classes_list.append(train_data_dict[j][i][1])
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                for i in x_batch:
                    image = mpimg.imread('train/' + i)
                    image = random_crop(image,image_size)
                    if(np.random.rand()<0.5):
                        image = np.rot90(image,1,(0,1))
                    image_crop_list.append(image)
                    del image
                    #del aug_image_recrop
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [14]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for start in range(0, len(valid_data), batch_size):
            image_crop_list = []
            image_classes_list = []
            end = min(start + batch_size, len(valid_data)) 
    
            x_batch = [valid_data[i][0] for i in range(start,end)]
            
            for index,i in enumerate(x_batch):
                image = mpimg.imread('../validation_224/' + i)
                if(len(image.shape)<3):
                        continue
                #rand_num = random.randint(0,7)
                
                #augmented_image = image_aug_valid(image,rand_num)
                #image = random_crop_valid(image,image_size)
                image_crop_list.append(image)
                image_classes_list.append(valid_data[start+index][1])
                
                del image
               
                #del aug_image_recrop
            
            
            image_crop_list = [x/1. for x in image_crop_list]
            #image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [15]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [17]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [10]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=2.5e-5),
             metrics=['accuracy'])

In [17]:
from pushbullet import Pushbullet
pb = Pushbullet('o.KiDDDXPuzV4qKbXh4Lywbgw1tK2oFfq1')

In [18]:
pushbullet_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: pb.push_note("epoch: "+str(epoch),"train_loss: "+str(logs['loss'])+"    val_loss"+str(logs['val_loss'])))

In [19]:
callbacks = [ModelCheckpoint(filepath='mobilenet_unalt_3.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
#EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./3-metrics_mobilenet_2.csv'),
TensorBoard(log_dir='logs_mobilenet', write_graph=True),
            pushbullet_callback]

In [ ]:
model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 20*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=valid_aug_generator,
                    validation_steps= 4*int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/100
1399/1400 [============================>.] - ETA: 0s - loss: 0.0589 - acc: 0.9837
Epoch 00001: val_loss improved from inf to 0.21349, saving model to mobilenet_unalt_3.hdf5
1400/1400 [==============================] - 545s 389ms/step - loss: 0.0588 - acc: 0.9837 - val_loss: 0.2135 - val_acc: 0.9437
Epoch 2/100
1399/1400 [============================>.] - ETA: 0s - loss: 0.0529 - acc: 0.9852
Epoch 00002: val_loss improved from 0.21349 to 0.19422, saving model to mobilenet_unalt_3.hdf5
1400/1400 [==============================] - 541s 387ms/step - loss: 0.0529 - acc: 0.9852 - val_loss: 0.1942 - val_acc: 0.9463
Epoch 3/100
1399/1400 [============================>.] - ETA: 0s - loss: 0.0491 - acc: 0.9864
Epoch 00003: val_loss improved from 0.19422 to 0.17961, saving model to mobilenet_unalt_3.hdf5
1400/1400 [==============================] - 555s 396ms/step - loss: 0.0491 - acc: 0.9865 - val_loss: 0.1796 - val_acc: 0.9500
Epoch 4/100
1399/1400 [============================>.] -

In [11]:
model.load_weights('mobilenet_unalt_3.hdf5')

In [23]:
model.evaluate_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

[0.089218300121530814, 0.9771167095793194]

In [21]:
preds_mobilenet = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

In [1]:
# validation set predictions - 224

In [22]:
f = defaultdict(list)
for i,j in enumerate(validation_generator.filenames):
    f[j.split('/')[-1][2:]].append(i)
    

In [24]:
result_filenames=[]
predicted_classes=[]
for i in validation_filenames:
    x = np.mean(preds_mobilenet[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [23]:
validation_filenames = np.load('filenames_list.npy')

In [26]:
np.save('mobilenet_preds_224.npy',predicted_classes)

In [28]:
np.save('filenames_list.npy',validation_generator.filenames)
np.save('target_classes.npy',validation_generator.classes)

In [26]:
np.save('mobilenet_preds.npy',preds_mobilenet)

In [22]:
predictions_valid = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)

In [23]:
validation_generator.class_indices

{'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [41]:
np.save('predictions.npy',predictions_valid)
np.save('correct_classes.npy',validation_generator.classes)

In [24]:
true_positive = np.zeros(10)
false_positive = np.zeros(10)
true_negative = np.zeros(10)
false_negative = np.zeros(10)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [25]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)


true_positive
[ 97.  98.  67.  68.  82.  73.  97.  80.  86.  89.]
false_positive
[ 6.  4.  3.  4.  1.  3.  4.  1.  4.  8.]
false_negative
[ 1.  1.  8.  9.  4.  3.  2.  6.  2.  2.]
Total
[ 98.  99.  75.  77.  86.  76.  99.  86.  88.  91.]


In [12]:
#test set predictions - 224

In [12]:
filenames_list = []
images_list = []
for i in os.listdir('test_224/temp/'):
    img = Image.open('test_224/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    

In [13]:
images_list = [preprocess_input(x) for x in images_list]
new_array_arr = np.array(images_list)
new_array_arr.shape

(13200, 224, 224, 3)

In [14]:
test_preds_224 = model.predict(new_array_arr)

In [15]:
f = defaultdict(list)
for i,j in enumerate(filenames_list):
    f[j[2:]].append(i)

In [16]:
test_filenames = np.load('test_filenames.npy')

In [17]:
result_filenames=[]
predicted_classes=[]
for i in test_filenames:
    x = np.mean(test_preds_224[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [18]:
np.save('mobilenet_test_preds_224.npy',predicted_classes)

In [2]:
# test set predictions - 512

In [18]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    

In [19]:
images_list = [preprocess_input(x) for x in images_list]

In [20]:
new_array_arr = np.array(images_list)


In [21]:
new_array_arr.shape

(2640, 512, 512, 3)

In [22]:
test_preds = model.predict(new_array_arr)

In [23]:
test_preds[0]

array([  4.17633884e-04,   8.13390288e-05,   9.98732269e-01,
         4.84149241e-05,   1.61176664e-04,   3.23370041e-05,
         5.92375109e-05,   1.19121309e-04,   2.59967608e-04,
         8.85234404e-05], dtype=float32)

In [24]:
np.save('mobilenet_test_preds.npy',test_preds)

In [58]:
f = defaultdict(list)
for i,j in enumerate(filenames_list):
    f[j[2:]].append(i)

In [60]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}

In [30]:
for index,i in enumerate(result_filenames):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]=''

In [65]:
submission = pd.DataFrame({'fname':result_filenames,'camera':predicted_classes})
submission.to_csv('submission_resnet_v2_ext_data_moto_x.csv', encoding="utf8", index=False)

In [66]:
from IPython.display import FileLink
FileLink('submission_resnet_v2_ext_data_moto_x.csv')

/home/janardhan/camera_model/submission_resnet_v2_ext_data_moto_x.csv

In [ ]:
# only densenet - manip+unalt - Private LB score - 97.22